#### This is my Work In Progress File. 

The code blocks in this file initially ran, but after making some changes to the code (for example implementing the one-hot encoding, or selecting the most important features first, and changing the dataset etc.), the snippets gave me error. I will store my wip here to explore later. 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.ensemble import VotingRegressor, RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, mean_squared_error
import xgboost as xgb
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.optimizers import Adam

In [ ]:
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.model_selection import GridSearchCV

# Define a function to evaluate the model
def evaluate_model(model, X_train, X_test, y_train, y_test):
    # Train the model
    model.fit(X_train, y_train)
    # Make predictions
    y_pred = model.predict(X_test)
    # Transform back to the original scale
    y_test_original = np.exp(y_test)
    y_pred_original = np.exp(y_pred)
    # Evaluate the model
    mae = mean_absolute_error(y_test_original, y_pred_original)
    mse = mean_squared_error(y_test_original, y_pred_original)
    return mae, mse, y_test_original, y_pred_original

# Define the models and their parameter grids
models = {
    'Ridge': Ridge(),
    'Lasso': Lasso(),
    'ElasticNet': ElasticNet()
}

params = {
    'Ridge': {'alpha': [0.1, 1.0, 10.0, 100.0]},
    'Lasso': {'alpha': [0.0001, 0.001, 0.01, 0.1, 1.0]},
    'ElasticNet': {'alpha': [0.1, 1.0, 10.0, 100.0], 'l1_ratio': [0.1, 0.5, 0.9]}
}

# Perform Grid Search with Cross-Validation
best_models = {}
for model_name in models:
    print(f"Training {model_name}...")
    grid_search = GridSearchCV(models[model_name], params[model_name], cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)
    grid_search.fit(X_train, y_train)
    best_models[model_name] = grid_search.best_estimator_
    print(f"Best parameters for {model_name}: {grid_search.best_params_}")

# Evaluate the best models
results = {}
for model_name in best_models:
    mae, mse, y_test_original, y_pred_original = evaluate_model(best_models[model_name], X_train, X_test, y_train, y_test)
    results[model_name] = {'MAE': mae, 'MSE': mse}
    print(f"\n{model_name} Performance:")
    print(f"Mean Absolute Error (MAE): {mae}")
    print(f"Mean Squared Error (MSE): {mse}")

    # Plot actual vs predicted
    plt.figure(figsize=(10, 6))
    plt.scatter(y_test_original, y_pred_original, color=custom_palette[8])
    plt.plot([min(y_test_original), max(y_test_original)], [min(y_test_original), max(y_test_original)], 'r', linewidth=2)
    plt.xlabel('Actual SalePrice')
    plt.ylabel('Predicted SalePrice')
    plt.title(f'Actual vs Predicted SalePrice ({model_name})')
    plt.show()


**Revised Approach**
- **Include More Features: Add more relevant features to the model.**
- **Extensive Hyperparameter Tuning: Use a more comprehensive grid for hyperparameter tuning**

In [ ]:
# Log transformation for skewed numerical features
ames_df['SalePrice'] = np.log(ames_df['SalePrice'])
ames_df['GrLivArea'] = np.log(ames_df['GrLivArea'])
ames_df['MasVnrArea'] = np.log1p(ames_df['MasVnrArea'])

# Creating a binary variable 'HasBsmt'
ames_df['HasBsmt'] = 0
ames_df.loc[ames_df['TotalBsmtSF'] > 0, 'HasBsmt'] = 1

# Applying log transformation to 'TotalBsmtSF' for non-zero values
ames_df.loc[ames_df['HasBsmt'] == 1, 'TotalBsmtSF'] = np.log(ames_df.loc[ames_df['HasBsmt'] == 1, 'TotalBsmtSF'])

# Convert categorical variables into dummy variables
ames_df = pd.get_dummies(ames_df, drop_first=True)

# Separate the target variable and features
X = ames_df.drop('SalePrice', axis=1)
y = ames_df['SalePrice']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale numerical features
scaler = StandardScaler()
X_train[X_train.select_dtypes(include=[np.number]).columns] = scaler.fit_transform(X_train.select_dtypes(include=[np.number]))
X_test[X_test.select_dtypes(include=[np.number]).columns] = scaler.transform(X_test.select_dtypes(include=[np.number]))

# Define the hyperparameter grid for Ridge, Lasso, and ElasticNet
param_grid = {
    'ridge': {'alpha': [0.1, 1, 10, 100, 200]},
    'lasso': {'alpha': [0.001, 0.01, 0.1, 1, 10]},
    'elasticnet': {'alpha': [0.1, 1, 10, 100], 'l1_ratio': [0.2, 0.5, 0.7, 0.9]}
}

# Define models
models = {
    'ridge': Ridge(),
    'lasso': Lasso(),
    'elasticnet': ElasticNet()
}

# Perform hyperparameter tuning and evaluation
best_models = {}
for model_name, model in models.items():
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid[model_name], cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)
    grid_search.fit(X_train, y_train)
    best_models[model_name] = grid_search.best_estimator_
    y_pred = grid_search.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    print(f"{model_name.capitalize()} Performance:")
    print(f"Best Parameters: {grid_search.best_params_}")
    print(f"Mean Absolute Error (MAE): {mae}")
    print(f"Mean Squared Error (MSE): {mse}")
    plt.figure(figsize=(10, 6))
    plt.scatter(y_test, y_pred, color='orange')
    plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], 'r', linewidth=2)
    plt.xlabel('Actual SalePrice')
    plt.ylabel('Predicted SalePrice')
    plt.title(f'Actual vs Predicted SalePrice ({model_name.capitalize()})')
    plt.show()

- **Mean Absolute Error (MAE):** The MAE represents the average absolute difference between the predicted values and the actual values. A lower MAE indicates better model performance. An MAE of 0.0737 suggests that, on average, the model's predictions are very close to the actual values.

- **Mean Squared Error (MSE):** The MSE is the average of the squared differences between predicted values and actual values.

- Like MAE, a lower MSE indicates better performance. An MSE of 0.0102 indicates that the model's predictions have a small error variance.

#### Comparison to Previous Results:

These metrics are notably better compared to the initial baseline model and other regression models we've tried. 

The MAE is significantly lower, indicating more accurate predictions.
The MSE is also much lower, suggesting less variability in the prediction errors.

- **Visualization**

The scatter plot of actual vs. predicted SalePrice for the ElasticNet model visually show the improvements, indicating how closely the predictions align with the actual values. A tighter cluster around the diagonal line (where predicted equals actual) confirm the model's accuracy.

#### ElasticNet:

The ElasticNet model has the highest MAE and MSE among the three models. This indicates it has the least accuracy and higher error variance.
Lasso:

The Lasso model performs better than ElasticNet with a lower MAE and MSE. This suggests it provides more accurate predictions with less variability in errors.
Ridge:

The Ridge model outperforms both ElasticNet and Lasso in terms of MAE and MSE. It has the lowest MAE and MSE, indicating the highest accuracy and least error variance among the three models.

#### 1. Cross-Validation
We'll perform k-fold cross-validation to ensure that the performance metrics are consistent across different subsets of the data.

In [ ]:
from sklearn.model_selection import cross_val_score

# Define the models
ridge_model = Ridge(alpha=10)
lasso_model = Lasso(alpha=0.001)
elasticnet_model = ElasticNet(alpha=0.1, l1_ratio=0.2)

# Perform 10-fold cross-validation
ridge_cv_scores = cross_val_score(ridge_model, X, y, cv=10, scoring='neg_mean_absolute_error')
lasso_cv_scores = cross_val_score(lasso_model, X, y, cv=10, scoring='neg_mean_absolute_error')
elasticnet_cv_scores = cross_val_score(elasticnet_model, X, y, cv=10, scoring='neg_mean_absolute_error')

# Print the cross-validation results
print(f"Ridge CV Mean Absolute Error: {-ridge_cv_scores.mean()} (+/- {ridge_cv_scores.std() * 2})")
print(f"Lasso CV Mean Absolute Error: {-lasso_cv_scores.mean()} (+/- {lasso_cv_scores.std() * 2})")
print(f"ElasticNet CV Mean Absolute Error: {-elasticnet_cv_scores.mean()} (+/- {elasticnet_cv_scores.std() * 2})")

#### 2. Feature Analysis

We'll analyze feature importance to understand which features contribute most to the model’s predictions.

In [ ]:
# Train the Ridge model on the full dataset
ridge_model.fit(X, y)

# Get feature importance (coefficients)
ridge_feature_importances = pd.Series(ridge_model.coef_, index=X.columns)

# Sort feature importances
ridge_feature_importances = ridge_feature_importances.sort_values(ascending=False)

# Plot feature importances
plt.figure(figsize=(12, 8))
ridge_feature_importances.head(20).plot(kind='bar')
plt.title('Feature Importances (Ridge Regression)')
plt.show()

In [ ]:
# Define the features to use
selected_features = ['OverallQual', 'GrLivArea', '1stFlrSF', 'TotalBsmtSF', 'GarageCars', 'GarageArea',
                     'YearBuilt', 'ExterQual_TA', 'BsmtFinSF1', 'GarageYrBlt', 'FullBath', 'YearRemodAdd',
                     'LotArea', 'MasVnrArea', 'KitchenQual_TA', 'Fireplaces', 'TotRmsAbvGrd', '2ndFlrSF',
                     'GarageFinish_Unf', 'BsmtQual_TA', 'OpenPorchSF', 'Foundation_PConc', 'Neighborhood_NridgHt',
                     'LotFrontage', 'BsmtQual_Gd', 'ExterQual_Gd', 'WoodDeckSF', 'KitchenQual_Gd', 'BsmtUnfSF',
                     'GarageType_Detchd', 'HalfBath', 'BsmtExposure_Gd', 'RoofStyle_Gable', 'BedroomAbvGr',
                     'MSSubClass', 'RoofStyle_Hip', 'BsmtFullBath', 'Foundation_CBlock', 'BsmtFinType1_GLQ', 
                     'OverallCond']

# Define features (X) and target (y)
X = ames_df[selected_features]
y = ames_df['SalePrice']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Ridge Regression
ridge_params = {'alpha': [0.1, 1.0, 10, 100]}
ridge_model = GridSearchCV(Ridge(), ridge_params, cv=5)
ridge_model.fit(X_train, y_train)
best_ridge_params = ridge_model.best_params_

ridge_pred = ridge_model.predict(X_test)
ridge_mae = mean_absolute_error(y_test, ridge_pred)
ridge_mse = mean_squared_error(y_test, ridge_pred)

print(f"Ridge Performance:\nBest Parameters: {best_ridge_params}\nMean Absolute Error (MAE): {ridge_mae}\nMean Squared Error (MSE): {ridge_mse}")



# ElasticNet Regression
elasticnet_params = {'alpha': [0.01, 0.1, 1.0], 'l1_ratio': [0.1, 0.5, 0.9]}
elasticnet_model = GridSearchCV(ElasticNet(), elasticnet_params, cv=5)
elasticnet_model.fit(X_train, y_train)
best_elasticnet_params = elasticnet_model.best_params_

elasticnet_pred = elasticnet_model.predict(X_test)
# Run Lasso Regression
lasso_model.fit(X_train, y_train)
lasso_pred = lasso_model.predict(X_test)

# Plot actual vs predicted for each model
plt.figure(figsize=(12, 8))
plt.scatter(y_test, ridge_pred, color='blue', label='Ridge Predictions')
plt.scatter(y_test, lasso_pred, color='green', label='Lasso Predictions')
plt.scatter(y_test, elasticnet_pred, color='orange', label='ElasticNet Predictions')
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], 'r', linewidth=2)
plt.xlabel('Actual SalePrice')
plt.ylabel('Predicted SalePrice')
plt.title('Actual vs Predicted SalePrice')
plt.legend()
plt.show()

In [ ]:
# Define a scaler
scaler = StandardScaler()

# Define the models
ridge = Ridge(max_iter=10000)
lasso = Lasso(max_iter=10000)
elasticnet = ElasticNet(max_iter=10000)

# Create pipelines for each model
ridge_pipeline = Pipeline([
    ('scaler', scaler),
    ('model', ridge)
])

lasso_pipeline = Pipeline([
    ('scaler', scaler),
    ('model', lasso)
])

elasticnet_pipeline = Pipeline([
    ('scaler', scaler),
    ('model', elasticnet)
])

# Define parameter grids
ridge_params = {'model__alpha': [0.01, 0.1, 1.0, 10, 100]}
lasso_params = {'model__alpha': [0.0001, 0.001, 0.01, 0.1, 1]}
elasticnet_params = {'model__alpha': [0.01, 0.1, 1.0], 'model__l1_ratio': [0.1, 0.5, 0.9]}

# Grid search for each model
ridge_search = GridSearchCV(ridge_pipeline, ridge_params, cv=5)
lasso_search = GridSearchCV(lasso_pipeline, lasso_params, cv=5)
elasticnet_search = GridSearchCV(elasticnet_pipeline, elasticnet_params, cv=5)

# Fit and evaluate Ridge
ridge_search.fit(X_train, y_train)
best_ridge_params = ridge_search.best_params_
ridge_pred = ridge_search.predict(X_test)
ridge_mae = mean_absolute_error(y_test, ridge_pred)
ridge_mse = mean_squared_error(y_test, ridge_pred)

print(f"Ridge Performance:\nBest Parameters: {best_ridge_params}\nMean Absolute Error (MAE): {ridge_mae}\nMean Squared Error (MSE): {ridge_mse}")

# Fit and evaluate Lasso
lasso_search.fit(X_train, y_train)
best_lasso_params = lasso_search.best_params_
lasso_pred = lasso_search.predict(X_test)
lasso_mae = mean_absolute_error(y_test, lasso_pred)
lasso_mse = mean_squared_error(y_test, lasso_pred)

print(f"Lasso Performance:\nBest Parameters: {best_lasso_params}\nMean Absolute Error (MAE): {lasso_mae}\nMean Squared Error (MSE): {lasso_mse}")

# Fit and evaluate ElasticNet
elasticnet_search.fit(X_train, y_train)
best_elasticnet_params = elasticnet_search.best_params_
elasticnet_pred = elasticnet_search.predict(X_test)
elasticnet_mae = mean_absolute_error(y_test, elasticnet_pred)
elasticnet_mse = mean_squared_error(y_test, elasticnet_pred)

print(f"ElasticNet Performance:\nBest Parameters: {best_elasticnet_params}\nMean Absolute Error (MAE): {elasticnet_mae}\nMean Squared Error (MSE): {elasticnet_mse}")

# Plot actual vs predicted for each model
plt.figure(figsize=(18, 5))

plt.subplot(1, 3, 1)
plt.scatter(y_test, ridge_pred, color='blue', label='Ridge Predictions')
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], 'r', linewidth=2)
plt.xlabel('Actual SalePrice')
plt.ylabel('Predicted SalePrice')
plt.title('Ridge Regression')
plt.legend()

plt.subplot(1, 3, 2)
plt.scatter(y_test, lasso_pred, color='green', label='Lasso Predictions')
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], 'r', linewidth=2)
plt.xlabel('Actual SalePrice')
plt.ylabel('Predicted SalePrice')
plt.title('Lasso Regression')
plt.legend()

plt.subplot(1, 3, 3)
plt.scatter(y_test, elasticnet_pred, color='orange', label='ElasticNet Predictions')
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], 'r', linewidth=2)
plt.xlabel('Actual SalePrice')
plt.ylabel('Predicted SalePrice')
plt.title('ElasticNet Regression')
plt.legend()

plt.tight_layout()
plt.show()

XGBoost V0

In [ ]:
import xgboost as xgb

# Define the model
xgboost_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

# Define parameter grid
xgboost_params = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0]
}

# Perform grid search
xgboost_search = GridSearchCV(xgboost_model, xgboost_params, cv=5)
xgboost_search.fit(X_train, y_train)

# Best parameters
best_xgboost_params = xgboost_search.best_params_

# Make predictions
xgboost_pred = xgboost_search.predict(X_test)

# Evaluate the model
xgboost_mae = mean_absolute_error(y_test, xgboost_pred)
xgboost_mse = mean_squared_error(y_test, xgboost_pred)

print(f"XGBoost Performance:\nBest Parameters: {best_xgboost_params}\nMean Absolute Error (MAE): {xgboost_mae}\nMean Squared Error (MSE): {xgboost_mse}")


#### Ensemble (Voting Regressor)

In [ ]:
from sklearn.ensemble import VotingRegressor

# Define the ensemble model
ensemble_model = VotingRegressor(estimators=[
    ('ridge', ridge_search.best_estimator_),
    ('lasso', lasso_search.best_estimator_),
    ('elasticnet', elasticnet_search.best_estimator_),
    ('xgboost', xgboost_search.best_estimator_)
])

# Fit the model
ensemble_model.fit(X_train, y_train)

# Make predictions
ensemble_pred = ensemble_model.predict(X_test)

# Evaluate the model
ensemble_mae = mean_absolute_error(y_test, ensemble_pred)
ensemble_mse = mean_squared_error(y_test, ensemble_pred)

print(f"Ensemble Performance:\nMean Absolute Error (MAE): {ensemble_mae}\nMean Squared Error (MSE): {ensemble_mse}")


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.ensemble import VotingRegressor, RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, mean_squared_error
import xgboost as xgb
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.optimizers import Adam

# Load the dataset
file_path = '../data/AmesData.csv'
ames_df = pd.read_csv(file_path)

# Replace blanks in 'MasVnrArea' with 0
ames_df['MasVnrArea'].replace(np.nan, 0, inplace=True)

# Apply log transformation to 'SalePrice'
ames_df['SalePrice'] = np.log(ames_df['SalePrice'])

# Create binary variable 'HasBsmt'
ames_df['HasBsmt'] = 0
ames_df.loc[ames_df['TotalBsmtSF'] > 0, 'HasBsmt'] = 1

# Apply log transformation to 'TotalBsmtSF' for non-zero values
ames_df.loc[ames_df['HasBsmt'] == 1, 'TotalBsmtSF'] = np.log(ames_df.loc[ames_df['HasBsmt'] == 1, 'TotalBsmtSF'])

# Convert categorical variables into dummy variables
ames_df = pd.get_dummies(ames_df, drop_first=True)

# Select the top 40 features from Random Forest (Example)
top_40_features = ['OverallQual', 'GrLivArea', '1stFlrSF', 'TotalBsmtSF', 'GarageCars', 'GarageArea',
                   'YearBuilt', 'ExterQual_TA', 'BsmtFinSF1', 'GarageYrBlt', 'FullBath', 'YearRemodAdd',
                   'LotArea', 'MasVnrArea', 'KitchenQual_TA', 'Fireplaces', 'TotRmsAbvGrd', '2ndFlrSF',
                   'GarageFinish_Unf', 'BsmtQual_TA', 'OpenPorchSF', 'Foundation_PConc', 'Neighborhood_NridgHt',
                   'LotFrontage', 'BsmtQual_Gd', 'ExterQual_Gd', 'WoodDeckSF', 'KitchenQual_Gd', 'BsmtUnfSF',
                   'GarageType_Detchd', 'HalfBath', 'BsmtExposure_Gd', 'RoofStyle_Gable', 'BedroomAbvGr',
                   'MSSubClass', 'RoofStyle_Hip', 'BsmtFullBath', 'Foundation_CBlock', 'BsmtFinType1_GLQ', 
                   'OverallCond']

# Define features (X) and target (y)
X = ames_df[top_40_features]
y = ames_df['SalePrice']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Ridge Regression
ridge_params = {'model__alpha': [0.1, 1.0, 10, 100]}
ridge_model = GridSearchCV(Ridge(), ridge_params, cv=5)
ridge_model.fit(X_train, y_train)
best_ridge_params = ridge_model.best_params_

ridge_pred = ridge_model.predict(X_test)
ridge_mae = mean_absolute_error(y_test, ridge_pred)
ridge_mse = mean_squared_error(y_test, ridge_pred)
print(f"Ridge Performance:\nBest Parameters: {best_ridge_params}\nMean Absolute Error (MAE): {ridge_mae}\nMean Squared Error (MSE): {ridge_mse}")

# Lasso Regression
lasso_params = {'model__alpha': [0.001, 0.01, 0.1, 1]}
lasso_model = GridSearchCV(Lasso(), lasso_params, cv=5)
lasso_model.fit(X_train, y_train)
best_lasso_params = lasso_model.best_params_

lasso_pred = lasso_model.predict(X_test)
lasso_mae = mean_absolute_error(y_test, lasso_pred)
lasso_mse = mean_squared_error(y_test, lasso_pred)
print(f"Lasso Performance:\nBest Parameters: {best_lasso_params}\nMean Absolute Error (MAE): {lasso_mae}\nMean Squared Error (MSE): {lasso_mse}")

# ElasticNet Regression
elasticnet_params = {'model__alpha': [0.01, 0.1, 1.0], 'model__l1_ratio': [0.1, 0.5, 0.9]}
elasticnet_model = GridSearchCV(ElasticNet(), elasticnet_params, cv=5)
elasticnet_model.fit(X_train, y_train)
best_elasticnet_params = elasticnet_model.best_params_

elasticnet_pred = elasticnet_model.predict(X_test)
elasticnet_mae = mean_absolute_error(y_test, elasticnet_pred)
elasticnet_mse = mean_squared_error(y_test, elasticnet_pred)
print(f"ElasticNet Performance:\nBest Parameters: {best_elasticnet_params}\nMean Absolute Error (MAE): {elasticnet_mae}\nMean Squared Error (MSE): {elasticnet_mse}")

# XGBoost
xgboost_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
xgboost_params = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0]
}
xgboost_search = GridSearchCV(xgboost_model, xgboost_params, cv=5)
xgboost_search.fit(X_train, y_train)
best_xgboost_params = xgboost_search.best_params_
xgboost_pred = xgboost_search.predict(X_test)
xgboost_mae = mean_absolute_error(y_test, xgboost_pred)
xgboost_mse = mean_squared_error(y_test, xgboost_pred)
print(f"XGBoost Performance:\nBest Parameters: {best_xgboost_params}\nMean Absolute Error (MAE): {xgboost_mae}\nMean Squared Error (MSE): {xgboost_mse}")

# Support Vector Regression (SVR)
svr_model = SVR()
svr_params = {
    'kernel': ['linear'],
    'C': [1, 10],
    'epsilon': [0.1]
}
# Use a smaller subset for tuning
X_train_small, _, y_train_small, _ = train_test_split(X_train, y_train, test_size=0.9, random_state=42)
svr_search = GridSearchCV(svr_model, svr_params, cv=3, n_jobs=-1)
svr_search.fit(X_train_small, y_train_small)
best_svr_params = svr_search.best_params_
svr_pred = svr_search.predict(X_test)
svr_mae = mean_absolute_error(y_test, svr_pred)
svr_mse = mean_squared_error(y_test, svr_pred)
print(f"SVR Performance:\nBest Parameters: {best_svr_params}\nMean Absolute Error (MAE): {svr_mae}\nMean Squared Error (MSE): {svr_mse}")

# Artificial Neural Network (ANN)
def build_ann_model():
    model = Sequential()
    model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='linear'))
    model.compile(loss='mean_squared_error', optimizer=Adam())
    return model

ann_model = KerasRegressor(build_fn=build_ann_model, epochs=100, batch_size=32, verbose=0)
ann_model.fit(X_train, y_train)
ann_pred = ann_model.predict(X_test)
ann_mae = mean_absolute_error(y_test, ann_pred)
ann_mse = mean_squared_error(y_test, ann_pred)
print(f"ANN Performance:\nMean Absolute Error (MAE): {ann_mae}\nMean Squared Error (MSE): {ann_mse}")

# Ensemble Model
ensemble_model = VotingRegressor(estimators=[
    ('ridge', ridge_model.best_estimator_),
    ('lasso', lasso_model.best_estimator_),
    ('elasticnet', elasticnet_model.best_estimator_),
    ('xgboost', xgboost_search.best_estimator_)
])
ensemble_model.fit(X_train, y_train)
ensemble_pred = ensemble_model.predict(X_test)
ensemble_mae = mean_absolute_error(y_test, ensemble_pred)
ensemble_mse = mean_squared_error(y_test, ensemble_pred)
print(f"Ensemble Performance:\nMean Absolute Error (MAE): {ensemble_mae}\nMean Squared Error (MSE): {ensemble_mse}")

# Summary of Results
results = {
    'Model': ['Ridge', 'Lasso', 'ElasticNet', 'XGBoost', 'SVR', 'ANN', 'Ensemble'],
    'MAE': [ridge_mae, lasso_mae, elasticnet_mae, xgboost_mae, svr_mae, ann_mae, ensemble_mae],
    'MSE': [ridge_mse, lasso_mse, elasticnet_mse, xgboost_mse, svr_mse, ann_mse, ensemble_mse]
}

results_df = pd.DataFrame(results)
print(results_df)

# Plot actual vs predicted for each model
plt.figure(figsize=(12, 8))
plt.scatter(y_test, ridge_pred, color='blue', label='Ridge Predictions')
plt.scatter(y_test, lasso_pred, color='green', label='Lasso Predictions')
plt.scatter(y_test, elasticnet_pred, color='orange', label='ElasticNet Predictions')
plt.scatter(y_test, xgboost_pred, color='purple', label='XGBoost Predictions')
plt.scatter(y_test, svr_pred, color='cyan', label='SVR Predictions')
plt.scatter(y_test, ann_pred, color='magenta', label='ANN Predictions')
plt.scatter(y_test, ensemble_pred, color='red', label='Ensemble Predictions')
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], 'black', linewidth=2)
plt.xlabel('Actual SalePrice')
plt.ylabel('Predicted SalePrice')
plt.title('Actual vs Predicted SalePrice')
plt.legend()
plt.show()
